# Introduction

Some handy examples showing how to automatically download files from Google drive, parse csv nd excel and load them into a Postgres database. The examples chosen are for the Save the Children stream of the DataKind 2022 DataDive.

- [project brief](https://docs.google.com/document/d/1TQ2TiGK_k8KEIUPzVb3ZSKaxzEBP9s9ZJBI2U_HeQ6U/edit#)
- [Google drive with data](https://drive.google.com/drive/folders/1G_CAhpb0xV9zRrV-T5ngA03x-IZ4tGKz)

## Setup

In [56]:
# If you don't have these installed, uncomment these lines and run
#%pip install gdown
#%pip install chardet
#%pip install sqlalchemy
#%pip install psycopg2-binary

In [13]:
import gdown
import os
import pandas as pd
import chardet

## Examples
### Downloading a folder from gdrive

In [9]:
folder_url = "https://drive.google.com/drive/folders/1G_CAhpb0xV9zRrV-T5ngA03x-IZ4tGKz?usp=share_link"
gdown.download_folder(folder_url, quiet=True, use_cookies=False)

['/Users/matthewharris/Desktop/git/STC_DataDive_Dec22/notebooks/dividor/Kenya/dbo-foodconsumptionscores.csv',
 '/Users/matthewharris/Desktop/git/STC_DataDive_Dec22/notebooks/dividor/Kenya/fts_incoming_funding_ken.csv',
 '/Users/matthewharris/Desktop/git/STC_DataDive_Dec22/notebooks/dividor/Kenya/fts_requirements_funding_cluster_ken.csv',
 '/Users/matthewharris/Desktop/git/STC_DataDive_Dec22/notebooks/dividor/Kenya/fts_requirements_funding_covid_ken.csv',
 '/Users/matthewharris/Desktop/git/STC_DataDive_Dec22/notebooks/dividor/Kenya/fts_requirements_funding_globalcluster_ken.csv',
 '/Users/matthewharris/Desktop/git/STC_DataDive_Dec22/notebooks/dividor/Kenya/fts_requirements_funding_ken.csv',
 '/Users/matthewharris/Desktop/git/STC_DataDive_Dec22/notebooks/dividor/Kenya/health_ken.csv',
 '/Users/matthewharris/Desktop/git/STC_DataDive_Dec22/notebooks/dividor/Kenya/hotosm_ken_roads_lines_shp.zip',
 '/Users/matthewharris/Desktop/git/STC_DataDive_Dec22/notebooks/dividor/Kenya/KE_202207.zip',
 

## Reading files in a directory to pandas

This snippet will read all csv and xls files in a directory into pandas dataframes, create a list of them.

Just an example, needs a bit more logic to handle xlsx sheets which hold multiple tables, blank rows etc.

In [33]:
dir = "./Kenya"

# Array of pandas dataframes
dfs = {}

for f in os.listdir(dir):
    f = f"{dir}/{f}"
    if '.csv' in f:
        print(f"Loading csv file {f}")
        # Detect encoding
        with open(f, 'rb') as rawdata:
            r = chardet.detect(rawdata.read(100000))
        dfs[f] = pd.read_csv(f, encoding=r['encoding'])
    if '.xlsx' in f:
        print(f"Leading {f} ...")
        # Read all sheets in
        sheet_to_df_map = pd.read_excel(f, sheet_name=None)
        for sheet in sheet_to_df_map:
            dfs[f"{f} - {sheet}"] =  sheet_to_df_map[sheet]


Loading csv file ./Kenya/fts_requirements_funding_covid_ken.csv
Loading csv file ./Kenya/dbo-foodconsumptionscores.csv
Leading ./Kenya/per-capita-food-consumption-2017-and-2018.xlsx ...
Leading ./Kenya/retail-prices-for-dry-beans-2019-per-kg.xlsx ...
Leading ./Kenya/value-of-recorded-marketed-agricultural-production-at-current-prices-2014-2018.xlsx ...
Loading csv file ./Kenya/wfp_food_prices_ken.csv
Loading csv file ./Kenya/fts_requirements_funding_cluster_ken.csv
Loading csv file ./Kenya/fts_incoming_funding_ken.csv
Loading csv file ./Kenya/health_ken.csv
Leading ./Kenya/ken_adminboundaries_tabulardata.xlsx ...
Loading csv file ./Kenya/fts_requirements_funding_ken.csv
Leading ./Kenya/ken_admpop_2019.xlsx ...
Loading csv file ./Kenya/ken_admpop_adm1_2019.csv
Loading csv file ./Kenya/ken_admpop_adm0_2019.csv
Leading ./Kenya/Kenya - IPC Analysis 2017-2022.xlsx ...
Leading ./Kenya/retail-prices-for-dry-beans-2018-per-kg.xlsx ...
Loading csv file ./Kenya/fts_requirements_funding_globalclu

/Users/matthewharris/opt/miniconda3/envs/stc_analysis/lib/python3.9/site-packages/openpyxl/worksheet/_reader.py:312: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)
/Users/matthewharris/opt/miniconda3/envs/stc_analysis/lib/python3.9/site-packages/openpyxl/worksheet/_reader.py:312: UserWarning: Conditional Formatting extension is not supported and will be removed
  warn(msg)


In [34]:
print(f"\n\nHere are the shapes of the dataframes we just loaded from directory {dir} ... \n\n")
for key in dfs:
    df = dfs[key]
    print(f"{key} >> {df.shape}")

print("Done")



Here are the shapes of the dataframes we just loaded from directory ./Kenya ... 


./Kenya/fts_requirements_funding_covid_ken.csv >> (3, 13)
./Kenya/dbo-foodconsumptionscores.csv >> (885, 15)
./Kenya/per-capita-food-consumption-2017-and-2018.xlsx - Sheet1 >> (21, 4)
./Kenya/retail-prices-for-dry-beans-2019-per-kg.xlsx - Prices for Dry Beans, 2019 >> (101, 4)
./Kenya/value-of-recorded-marketed-agricultural-production-at-current-prices-2014-2018.xlsx - Sheet1 >> (32, 8)
./Kenya/wfp_food_prices_ken.csv >> (14451, 14)
./Kenya/fts_requirements_funding_cluster_ken.csv >> (140, 12)
./Kenya/fts_incoming_funding_ken.csv >> (183, 37)
./Kenya/health_ken.csv >> (9412, 6)
./Kenya/ken_adminboundaries_tabulardata.xlsx - Admin2 >> (290, 16)
./Kenya/ken_adminboundaries_tabulardata.xlsx - Admin1 >> (47, 14)
./Kenya/ken_adminboundaries_tabulardata.xlsx - Admin0 >> (1, 12)
./Kenya/fts_requirements_funding_ken.csv >> (47, 12)
./Kenya/ken_admpop_2019.xlsx - ken_admpop_ADM0_2019 >> (1, 71)
./Kenya/ken_admp

## Loading pandas dataframes into a postgres database

Roughcode for how one might load all the dataframes into a SQL database. You can install postgres locally, or use docker and run [this](https://hub.docker.com/_/postgres).

In [75]:
from sqlalchemy import create_engine  
import sqlalchemy

# Set these for your Postgres instance
host   = 'localhost' 
dbname = 'postgres'
user   = 'postgres'
pwd    = 'postgres' #os.getenv['PWD'] # Don't save passwords in notebooks - that's naughty! :) - use environment variables. :)
port   = '5432'

engine = create_engine('postgresql+psycopg2://' + user + ':' + pwd + '@' + host +':' + port + '/' + dbname)

for k in dfs:
    df = dfs[k]
    table = k.replace('.','').replace(' ','_').replace('-','_').replace('/','').replace(',','')[0:62]
    print(f"Loading into database table {table} ...")
    df.to_sql(table, engine, if_exists='replace')

print("Done")

Loading into database table Kenyafts_requirements_funding_covid_kencsv ...
Loading into database table Kenyadbo_foodconsumptionscorescsv ...
Loading into database table Kenyaper_capita_food_consumption_2017_and_2018xlsx___Sheet1 ...
Loading into database table Kenyaretail_prices_for_dry_beans_2019_per_kgxlsx___Prices_for_ ...
Loading into database table Kenyavalue_of_recorded_marketed_agricultural_production_at_cur ...
Loading into database table Kenyawfp_food_prices_kencsv ...
Loading into database table Kenyafts_requirements_funding_cluster_kencsv ...
Loading into database table Kenyafts_incoming_funding_kencsv ...
Loading into database table Kenyahealth_kencsv ...
Loading into database table Kenyaken_adminboundaries_tabulardataxlsx___Admin2 ...
Loading into database table Kenyaken_adminboundaries_tabulardataxlsx___Admin1 ...
Loading into database table Kenyaken_adminboundaries_tabulardataxlsx___Admin0 ...
Loading into database table Kenyafts_requirements_funding_kencsv ...
Loading i

## Query from the database

In [73]:
query = ' SELECT * FROM public."Kenyakenya_3w_final_list_2017xlsx___drr"'
df = pd.read_sql(query, con=engine)
display(df)

,index,Unnamed: 0,Unnamed: 1
0,0,Baringo,WVI
1,1,Embu,Trocaire
2,2,Garissa,ADESO
3,3,Homa Bay,PLAN
4,4,Isiolo,ADESO
5,5,Kilifi,PLAN
6,6,Kisumu,PLAN
7,7,Kitui,Trocaire
8,8,Kwale,PLAN
9,9,Machakos,PLAN
